In [24]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('./concept_results.csv')
updated_gt = []
updated_out_wc = []
updated_out_woc = []

for index, row in df.iterrows():
    recording = row["Recording"]
    gt = str(row["Ground Truth"])
    out_wc = str(row["Extracted Output WC"])
    out_woc = str(row["Extracted Output WOC"])

    concept_arr = []

    for idx,concept in enumerate(out_wc.splitlines()):
        concept = concept.strip("()")
        concept = concept.lower()
        concept_arr.append(concept)
    
    out_wc = concept_arr
    updated_out_wc.append(out_wc)
    
    concept_arr = []

    for idx,concept in enumerate(out_woc.splitlines()):
        concept = concept.strip("()")
        concept = concept.lower()
        concept_arr.append(concept)
    
    out_woc = concept_arr
    updated_out_woc.append(out_woc)
    
    gt = gt.split(";")
    for idx,concept in enumerate(gt):
        concept = concept.strip("\r\n")
        concept = concept.lower()
        gt[idx] = concept
        # print(concept)
    updated_gt.append(gt)

df["Ground Truth"] = updated_gt
df["Extracted Output WC"] = updated_out_wc
df["Extracted Output WOC"] = updated_out_woc


#### TP, FP, FN Extraction for Chunked Audio

In [26]:
true_positives = []
false_positives = []
false_negatives = []

for index, row in df.iterrows():
    gt = row["Ground Truth"]
    out_wc = row["Extracted Output WC"]
    
    intersection = (set(gt) & set(out_wc))
    tp = intersection #extracted concepts that are true
    
    fp = set(out_wc) - tp #extracted concepts that are false
    
    
    fn = set(gt) - set(out_wc) #missed extracting true concepts
    
    fn  = [i for i in fn if i]

    
    true_positives.append(list(tp))
    false_positives.append(list(fp))
    false_negatives.append(list(fn))
    
    print("true_positives: ",tp)
    print("false_positives: ",fp)
    print("false_negatives: ",fn)
    print("")
    
    # print("gt: ",gt)
    # print("pred: ",out)
    
df["True Positives - WC"] = true_positives
df["False Positives - WC"] = false_positives
df["False Negatives - WC"] = false_negatives

# print(df)
# df.to_csv("updated_concept_results.csv")

true_positives:  {'loss of consciousness, true, unconscious', 'age, true, 55', 'spo2, true, 94', 'resp, true, 5', 'pulse, true, 118', 'bradypnea, true, 5', 'tachycardia, true, 118', 'gender, true, male'}
false_positives:  {'bp, true, 205/19', 'hypertension, true, 205'}
false_negatives:  ['glucose, true, 126', 'bp, true, 205/119', 'pale, true, pale', 'hypertension, true, 205/119']

true_positives:  {'shortness of breath, true, shortness of breath', 'wheezing, true, wheezing', 'gender, true, female', 'age, true, 51', 'hypertension, true, hypertension'}
false_positives:  {'shortness of breath, true, of shortness breath', 'pain, false, headache', 'headache, false, headache', 'dizziness, false, dizziness'}
false_negatives:  ['dizziness, true, dizziness', 'dysrhythmia, false, ekg', 'headache, true, headache', 'ekg, true, ekg', 'chest pain, true, chest fullness']

true_positives:  {'abdominal pain, true, abdominal pain', 'resp, true, 18', 'spo2, true, 100', 'chest pain, false, chest pain', 'g

#### TP, FP, FN Extraction for Entire Audio Without Chunking

In [27]:
true_positives = []
false_positives = []
false_negatives = []

for index, row in df.iterrows():
    gt = row["Ground Truth"]
    out_woc = row["Extracted Output WOC"]
    
    intersection = (set(gt) & set(out_woc))
    tp = intersection #extracted concepts that are true
    
    fp = set(out_woc) - tp #extracted concepts that are false
    
    
    fn = set(gt) - set(out_woc) #missed extracting true concepts
    
    fn  = [i for i in fn if i]

    
    true_positives.append(list(tp))
    false_positives.append(list(fp))
    false_negatives.append(list(fn))
    
    print("true_positives: ",tp)
    print("false_positives: ",fp)
    print("false_negatives: ",fn)
    print("")
    
    # print("gt: ",gt)
    # print("pred: ",out)
    
df["True Positives - WOC"] = true_positives
df["False Positives - WOC"] = false_positives
df["False Negatives - WOC"] = false_negatives

# print(df)
df.to_csv("updated_concept_results.csv")

true_positives:  {'loss of consciousness, true, unconscious', 'age, true, 55', 'bp, true, 205/119', 'spo2, true, 94', 'pulse, true, 118', 'tachycardia, true, 118', 'gender, true, male'}
false_positives:  {'hypertension, true, 205', 'resp, true, breathing'}
false_negatives:  ['glucose, true, 126', 'resp, true, 5', 'bradypnea, true, 5', 'pale, true, pale', 'hypertension, true, 205/119']

true_positives:  {'age, true, 51', 'dizziness, true, dizziness', 'gender, true, female'}
false_positives:  {'pain, false, headache', 'lightheadedness, true, dizziness', 'hypertension, false, hypertension', 'wheezing, true, lungs sound', 'headache, false, headache', 'shortness of breath, true, of shortness breath'}
false_negatives:  ['shortness of breath, true, shortness of breath', 'headache, true, headache', 'wheezing, true, wheezing', 'ekg, true, ekg', 'dysrhythmia, false, ekg', 'hypertension, true, hypertension', 'chest pain, true, chest fullness']

true_positives:  {'gcs, true, 15', 'abdominal pain, 

#### F1, Precision, Recall Calculation for Chunked Audio

In [28]:
concept_precision = []
concept_recall = []
concept_f1 = []

for index, row in df.iterrows():
    tp = row["True Positives - WC"]
    fp = row["False Positives - WC"]
    fn = row["False Negatives - WC"]
    
    precision = len(tp)/(len(tp)+len(fp))
    recall = len(tp)/(len(tp)+len(fn))
    f1 = 2*(precision*recall)/(precision+recall)
    
    concept_precision.append(precision)
    concept_recall.append(recall)
    concept_f1.append(f1)
    
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("F1: ",f1)
    
df["F1 - WC"] = concept_f1
df["Precision - WC"] = concept_precision
df["Recall - WC"] = concept_recall

df.to_csv("updated_concept_results.csv")


Precision:  0.8
Recall:  0.6666666666666666
F1:  0.7272727272727272
Precision:  0.5555555555555556
Recall:  0.5
F1:  0.5263157894736842
Precision:  0.4666666666666667
Recall:  0.4375
F1:  0.45161290322580644
Precision:  0.625
Recall:  0.4166666666666667
F1:  0.5
Precision:  0.3
Recall:  0.3
F1:  0.3
Precision:  0.75
Recall:  0.5454545454545454
F1:  0.631578947368421
Precision:  0.42857142857142855
Recall:  0.375
F1:  0.39999999999999997
Precision:  0.5
Recall:  0.2857142857142857
F1:  0.36363636363636365
Precision:  0.7
Recall:  0.4375
F1:  0.5384615384615384
Precision:  0.5
Recall:  0.3888888888888889
F1:  0.43750000000000006
Precision:  0.4117647058823529
Recall:  0.3888888888888889
F1:  0.39999999999999997
Precision:  0.2
Recall:  0.15789473684210525
F1:  0.17647058823529413


#### F1, Precision, Recall Calculation for Entire Audio Without Chunking

In [29]:
concept_precision = []
concept_recall = []
concept_f1 = []

for index, row in df.iterrows():
    tp = row["True Positives - WOC"]
    fp = row["False Positives - WOC"]
    fn = row["False Negatives - WOC"]
    
    precision = len(tp)/(len(tp)+len(fp))
    recall = len(tp)/(len(tp)+len(fn))
    f1 = 2*(precision*recall)/(precision+recall)
    
    concept_precision.append(precision)
    concept_recall.append(recall)
    concept_f1.append(f1)
    
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("F1: ",f1)
    
df["F1 - WOC"] = concept_f1
df["Precision - WOC"] = concept_precision
df["Recall - WOC"] = concept_recall

df.to_csv("updated_concept_results.csv")


Precision:  0.7777777777777778
Recall:  0.5833333333333334
F1:  0.6666666666666666
Precision:  0.3333333333333333
Recall:  0.3
F1:  0.3157894736842105
Precision:  0.6
Recall:  0.5625
F1:  0.5806451612903225
Precision:  0.8571428571428571
Recall:  0.5
F1:  0.631578947368421
Precision:  0.4
Recall:  0.2
F1:  0.26666666666666666
Precision:  0.625
Recall:  0.45454545454545453
F1:  0.5263157894736842
Precision:  0.42857142857142855
Recall:  0.375
F1:  0.39999999999999997
Precision:  0.23076923076923078
Recall:  0.21428571428571427
F1:  0.22222222222222224
Precision:  0.7777777777777778
Recall:  0.4375
F1:  0.56
Precision:  0.6153846153846154
Recall:  0.4444444444444444
F1:  0.5161290322580646
Precision:  0.6875
Recall:  0.6111111111111112
F1:  0.6470588235294118
Precision:  0.25
Recall:  0.21052631578947367
F1:  0.22857142857142856
